# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import hvplot.pandas
import pandas as pd
import numpy as np
import requests
import json
import os
import holoviews as hv
hv.extension('bokeh')


# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,delta,5.5000,6.0000,27.56,82,0,2.23,NG,1707778494
1,1,isafjordur,66.0755,-23.1240,-0.50,81,100,10.05,IS,1707778552
2,2,bilibino,68.0546,166.4372,-36.87,98,93,1.00,RU,1707778552
3,3,petropavlovsk-kamchatsky,53.0452,158.6483,-18.08,81,0,4.00,RU,1707778553
4,4,strathalbyn,-35.2667,138.9000,29.25,45,7,4.82,AU,1707778554


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [ ]:
# Configure the map plot
plot_city = city_data_df.hvplot.points(x='Lng', y='Lat', size = "Humidity", geo = True, tiles = "OSM", \
                                  frame_width = 800, frame_height = 600, color = "City")


# Display the map
hvplot.show(plot_city)

#hv.extension('bokeh')
#from bokeh.plotting import show
#show(hv.render(plot_city))


### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [3]:
ideal_df = city_data_df.loc[(city_data_df['Max Temp']>=21.03)\
                            &(city_data_df['Max Temp']<=26.27)\
                             &(city_data_df['Cloudiness']==0)\
                            &(city_data_df['Humidity']>=50),:]
print(len(ideal_df))

# Drop any rows with null values
ideal_df.dropna()

# Display sample data
ideal_df

10


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
9,9,wailua homesteads,22.0669,-159.3780,25.63,57,0,4.12,US,1707778033
48,48,east london,-33.0153,27.9116,22.47,84,0,8.35,ZA,1707778584
94,94,cabo san lucas,22.8909,-109.9124,24.78,59,0,4.39,MX,1707778429
134,134,port elizabeth,-33.9180,25.5701,22.40,88,0,1.34,ZA,1707778643
137,137,holualoa,19.6228,-155.9522,24.91,67,0,4.12,US,1707778645
155,155,piton saint-leu,-21.2196,55.3151,25.04,83,0,1.03,RE,1707778659
180,180,panaba,21.2833,-88.2667,25.24,84,0,5.41,MX,1707778680
240,240,port lincoln,-34.7333,135.8667,21.48,59,0,5.09,AU,1707778726
280,280,carnarvon,-24.8667,113.6333,24.13,83,0,8.45,AU,1707778752
429,429,rosarito,32.3333,-117.0333,21.07,56,0,5.45,MX,1707778808


### Step 3: Create a new DataFrame called `hotel_df`.

In [4]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df.copy()

hotel_df = hotel_df[["City","Country","Lat","Lng","Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,delta,NG,5.5000,6.0000,82,
1,isafjordur,IS,66.0755,-23.1240,81,
2,bilibino,RU,68.0546,166.4372,98,
3,petropavlovsk-kamchatsky,RU,53.0452,158.6483,81,
4,strathalbyn,AU,-35.2667,138.9000,45,
...,...,...,...,...,...,...
553,dalnegorsk,RU,44.5604,135.5701,67,
554,kosong,KR,38.3788,128.4676,77,
555,carmo do rio claro,BR,-20.9719,-46.1189,73,
556,pangody,RU,65.8500,74.4859,99,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [ ]:
# Set parameters to search for a hotel
radius = 10000
params = {'categories':'accommodation.hotel',\
         'filter':'',\
         'bias':'',\
         'apiKey':geoapify_key,\
         'limit':1}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = hotel_df.loc[index, "Lng"]
    latitude = hotel_df.loc[index, "Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{longitude},{latitude},{radius}'
    params["bias"] = f'proximity:{longitude},{latitude}'

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
delta - nearest hotel: Ekere Hotel
isafjordur - nearest hotel: Hótel Horn
bilibino - nearest hotel: No hotel found
petropavlovsk-kamchatsky - nearest hotel: ООО Постоялый двор
strathalbyn - nearest hotel: No hotel found
waitangi - nearest hotel: Hotel Chathams
vallenar - nearest hotel: Hotel del Marqués
verkhniy baskunchak - nearest hotel: No hotel found
trenggalek - nearest hotel: Atriaz
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
brooks - nearest hotel: No hotel found
mata-utu - nearest hotel: Gîte Oceania
anadyr - nearest hotel: Гостевой дом
avarua - nearest hotel: Paradise Inn
blackmans bay - nearest hotel: Villa Howden
west island - nearest hotel: Cocos Village Bungalows
kodiak - nearest hotel: Shelikof Lodge
suva - nearest hotel: Extension 68
vicuna mackenna - nearest hotel: No hotel found
bac lieu - nearest hotel: Khách sạn Bạc Liêu
prabumulih - nearest hotel: No hotel found
punta arenas - nearest hotel: Hotel Lacolet
campoverde - 

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

# Configure the map plot
plot_4 = hotel_df.hvplot.points('Lng',
            'Lat',
            geo=True, 
            tiles = 'OSM', 
            frame_width = 700, 
            frame_height = 500, 
            color='City',
            alpha=0.6,
            hover=True,
            hover_cols='all',
            use_index=False,
            s='Humidity',
            scale= 1.2)

# Display the map
plot_4